# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [2]:
class matrix:
    def __init__(self, a, b=None):
        if isinstance(a, list):
            self.data = a
            rlen = len(a[0])
            for r in a:
                if len(r) != rlen:
                    raise ValueError("rows not same size bruh")
            self.n = len(a)
            self.m = rlen
        else:
            self.n = a
            self.m = b
            self.data = [[0.0 for j in range(self.m)] for i in range(self.n)]

    def shape(self):
        return (self.n, self.m)

    def __getitem__(self, key):
        if isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        return self.data[key]

    def __setitem__(self, key, val):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = val
        else:
            self.data[key] = val

    def to_list(self):
        return [row[:] for row in self.data]

    def transpose(self):
        stuff = []
        for j in range(self.m):
            stuff.append([self.data[i][j] for i in range(self.n)])
        return matrix(stuff)

    def row(self, k):
        return matrix([self.data[k][:]])

    def column(self, k):
        return matrix([[self.data[i][k]] for i in range(self.n)])

    def block(self, n0, n1, m0, m1):
        blk = []
        for i in range(n0, n1):
            blk.append(self.data[i][m0:m1])
        return matrix(blk)

    def scalarmul(self, c):
        out = matrix(self.n, self.m)
        for i in range(self.n):
            for j in range(self.m):
                out[i, j] = self[i, j] * c
        return out

    def add(self, other):
        if self.shape() != other.shape():
            raise ValueError("sizes dont match")
        out = matrix(self.n, self.m)
        for i in range(self.n):
            for j in range(self.m):
                out[i, j] = self[i, j] + other[i, j]
        return out

    def sub(self, other):
        if self.shape() != other.shape():
            raise ValueError("sizes dont match")
        out = matrix(self.n, self.m)
        for i in range(self.n):
            for j in range(self.m):
                out[i, j] = self[i, j] - other[i, j]
        return out

    def element_mult(self, other):
        if self.shape() != other.shape():
            raise ValueError("sizes dont match")
        out = matrix(self.n, self.m)
        for i in range(self.n):
            for j in range(self.m):
                out[i, j] = self[i, j] * other[i, j]
        return out

    def mat_mult(self, other):
        if self.m != other.n:
            raise ValueError("bad sizes for mult")
        out = matrix(self.n, other.m)
        for i in range(self.n):
            for j in range(other.m):
                s = 0
                for k in range(self.m):
                    s += self[i, k] * other[k, j]
                out[i, j] = s
        return out

    def equals(self, other):
        if self.shape() != other.shape():
            return False
        for i in range(self.n):
            for j in range(self.m):
                if self[i, j] != other[i, j]:
                    return False
        return True

    def __add__(self, other):
        return self.add(other)

    def __sub__(self, other):
        return self.sub(other)

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        return self.mat_mult(other)

    def __rmul__(self, other):
        return self.scalarmul(other)

    def __eq__(self, other):
        return self.equals(other)


def constant(n, m, c):
    return matrix([[float(c) for j in range(m)] for i in range(n)])

def zeros(n, m):
    return constant(n, m, 0)

def ones(n, m):
    return constant(n, m, 1)

def eye(n):
    e = zeros(n, n)
    for i in range(n):
        e[i, i] = 1.0
    return e


print("testing init")
A = matrix(2, 2)
B = matrix([[1, 2], [3, 4]])

print(A.to_list())
print(B[0][1], B[1, 0])
print(B[0, 1])

print("shape:", B.shape())
print("transpose:", B.transpose().to_list())
print("row:", B.row(0).to_list())
print("col:", B.column(1).to_list())
print("block:", B.block(0, 2, 0, 1).to_list())

print("zeros:", zeros(2, 3).to_list())
print("ones:", ones(2, 2).to_list())
print("eye:", eye(2).to_list())

print("scalar mult:", (2 * B).to_list())
print("add:", (B + B).to_list())
print("sub:", (B - B).to_list())
print("elem mult:", B.element_mult(B).to_list())

A = matrix([[1, 2], [3, 4]])
B = matrix([[0, 1], [1, 0]])
C = matrix([[2, 0], [0, 2]])
I = eye(2)

print("(AB)C == A(BC):", (A * B * C) == (A * (B * C)))
print("A(B+C) == AB+AC:", A * (B + C) == (A * B + A * C))
print("AB != BA:", (A * B) != (B * A))
print("AI == A:", (A * I) == A)

testing init
[[0.0, 0.0], [0.0, 0.0]]
2 3
2
shape: (2, 2)
transpose: [[1, 3], [2, 4]]
row: [[1, 2]]
col: [[2], [4]]
block: [[1], [3]]
zeros: [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]
ones: [[1.0, 1.0], [1.0, 1.0]]
eye: [[1.0, 0.0], [0.0, 1.0]]
scalar mult: [[2, 4], [6, 8]]
add: [[2, 4], [6, 8]]
sub: [[0, 0], [0, 0]]
elem mult: [[1, 4], [9, 16]]
(AB)C == A(BC): True
A(B+C) == AB+AC: True
AB != BA: True
AI == A: True
